In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np

In [ ]:
load_data()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(     Unnamed: 0 IAPTus_Num Referral Date  Age_ReferralRequest_ReceivedDate  \
 0             1      24475    08/09/2018                          5.099020   
 1             2    24476_1    10/04/2019                          4.358899   
 2             3    24476_2    28/03/2021                          4.582576   
 3             4      24479    16/04/2019                          4.898979   
 4             5      24482    07/09/2018                          0.000000   
 ..          ...        ...           ...                               ...   
 723         724      24706           NaN                          0.000000   
 724         725      24721           NaN                          0.000000   
 725         727      24744           NaN                          0.000000   
 726         730      24658           NaN                          0.000000   
 727         737      24786           NaN                          0.000000   
 
      EthnicDescGroupCode  EthnicCategoryGroupShor

In [171]:
def rename_features(df): # non-questionnaire features

    df.rename(columns = {
        'Unnamed: 0': 'CaseID',
        'IAPTus_Num': 'ClientID',
        'Referral Date': 'ReferralDate',
        'Age_ReferralRequest_ReceivedDate': 'AgeAtReferralRequest',
        'EthnicDescGroupCode': 'EthnicCode',
        'EthnicCategoryGroupShortCode': 'EthnicCodeShort',
        'GenderIdentity': 'Gender',
        'SexualOrientationDesc': 'SexualOrientation',
        'EndDescGroupShort': 'Treated',
        'AllocatedCareProfNameCode': 'TherapistID',
        'JobTitleCode': 'ExperienceLevel',
        'Days to first assessment': 'DaystoAssessment',
        'Days to first treatment': 'DaystoTreatment',
        'CountOfAttendedCareContacts': 'CareContacts',
        'RecoveryDesc': 'Recovery',
        'ReliableRecoveryDesc': 'ReliableRecovery',
        'Date': 'DateOfQuestionnaire'},
        inplace = True)

    return df

# Functions

## 1.2 Preprocessing Functions

In [128]:
def load_data():

    drive.mount("/content/drive") # access google drive folder

    file_path = '/content/drive/MyDrive/Data/Dissertation_Data/'
    df = pd.read_csv(file_path + 'mental_health.csv', delimiter = ',') # csv
    raw_df = df.copy()

    return df, raw_df

In [129]:
def manage_df(df, name, option):

    diss_path = '/content/drive/MyDrive/Data/Dissertation_Data/'

    if option == 'save':
        df.to_csv(file_path + name + '.csv')
        print('Saved!')

    elif option == 'load':
        df = pd.read_csv(file_path + name + '.csv', index_col=0)
        return df

2.2 Non-Questionnaire Data Preprocessing

In [130]:
def rename_features(df): # non-questionnaire features

    df.rename(columns = {
        'Unnamed: 0': 'CaseID',
        'IAPTus_Num': 'ClientID',
        'Referral Date': 'ReferralDate',
        'Age_ReferralRequest_ReceivedDate': 'AgeAtReferralRequest',
        'EthnicDescGroupCode': 'EthnicCode',
        'EthnicCategoryGroupShortCode': 'EthnicCodeShort',
        'GenderIdentity': 'Gender',
        'SexualOrientationDesc': 'SexualOrientation',
        'EndDescGroupShort': 'Treated',
        'AllocatedCareProfNameCode': 'TherapistID',
        'JobTitleCode': 'ExperienceLevel',
        'Days to first assessment': 'DaystoAssessment',
        'Days to first treatment': 'DaystoTreatment',
        'CountOfAttendedCareContacts': 'CareContacts',
        'RecoveryDesc': 'Recovery',
        'ReliableRecoveryDesc': 'ReliableRecovery',
        'Date': 'DateOfQuestionnaire'},
        inplace = True)

    return df

In [131]:
def create_referral_count(df):
    def count_referrals(col):
        if '_1' in col:
            return 1
        elif '_2' in col:
            return 2
        elif '_3' in col:
            return 3
        elif '_4' in col:
            return 4
        elif '_5' in col:
            return 5
        else:
            return 1
    df.insert(2, "ReferralCount", df['ClientID'].apply(count_referrals)) # introduce next to ClientID
    return df

def clean_client_id(df):
    for text in ['_1', '_2', '_3', '_4']:
        df['ClientID'] = df['ClientID'].str.replace(text, '') # remove ending
    df['ClientID'] = pd.to_numeric(df['ClientID'])
    return df

def convert_features_to_datetime(df):
    df['ReferralDate'] = pd.to_datetime(df['ReferralDate'], format = '%d/%m/%Y')
    df['DateOfQuestionnaire'] = pd.to_datetime(df['DateOfQuestionnaire'], format = '%d/%m/%Y')
    return df

def convert_float_features_to_int(df):
    df['EthnicCode'] = df['EthnicCode'].astype('Int64') # Int deals with NaNs, int does not
    df['EthnicCodeShort'] = df['EthnicCodeShort'].astype('Int64')
    df['TherapistID'] = df['TherapistID'].astype('Int64')
    df['ExperienceLevel'] = df['ExperienceLevel'].astype('Int64')
    return df

In [132]:
def map_features(df):

    Gender_map = {
        'CHANGE ME': np.nan,
        'X': np.nan}
    df['Gender'] = df['Gender'].replace(Gender_map).astype('Int64')

    Treated_map = {
        'Seen and treated': 1,
        'Seen but not treated': 0}
    df['Treated'] = df['Treated'].replace(Treated_map).astype('Int64')

    ReliableChangeDesc_map = {
        'Reliable improvement': 2, # what about(-1, 0, 1)?
        'No reliable change': 1,
        'Reliable deterioration': 0,
        'Not applicable': np.nan}
    df['ReliableChangeDesc'] = df['ReliableChangeDesc'].replace(ReliableChangeDesc_map).astype('Int64')

    Recovery_map = {
        'At recovery': 1,
        'Not at recovery': 0,
        'Not applicable': np.nan}
    df['Recovery'] = df['Recovery'].replace(Recovery_map).astype('Int64')

    ReliableRecovery_map = {
        'Reliable recovery': 1,
        'No reliable recovery': 0,
        'Not applicable': np.nan}
    df['ReliableRecovery'] = df['ReliableRecovery'].replace(ReliableRecovery_map).astype('Int64')

    return df


In [133]:
def one_hot_encode_features(df):

    EndDesc_cols = pd.get_dummies(df['EndDesc'], prefix = 'EndDesc')
    EndDesc_index = df.columns.get_loc('EndDesc')
    df = pd.concat([df.iloc[:, :EndDesc_index + 1], EndDesc_cols, df.iloc[:, EndDesc_index + 1:]], axis = 1)
    df = df.drop(columns = ['EndDesc'])

    EndDescShort_cols = pd.get_dummies(df['EndDescShort'], prefix = 'EndDescShort')
    EndDescShort_index = df.columns.get_loc('EndDescShort')
    df = pd.concat([df.iloc[:, :EndDescShort_index + 1], EndDescShort_cols, df.iloc[:, EndDescShort_index + 1:]], axis = 1)
    df = df.drop(columns = ['EndDescShort'])

    return df

def convert_to_int_features(df):
    int_cols = ['SexualOrientation', 'DaystoAssessment', 'DaystoTreatment', 'CareContacts']
    for col in int_cols:
        df[col] = df[col].astype('Int64')
    return df

In [134]:
def plot_features(df):

    plot_cols = 4
    plot_rows = len(df.columns)//4 + 1

    plt.figure(figsize = (plot_cols*3, plot_rows*3))

    for i, col in enumerate(df.columns, start = 1):
        if df[col].dtype in ['int64', 'float64', 'Int64', 'datetime64[ns]', 'bool']:
            data = df[col].dropna()
            if df[col].dtype == 'bool':  # convert boolean to integers
                data = data.astype(int)
            plt.subplot(plot_rows, 4, i)
            plt.hist(data, bins = 20, color='darkgrey', edgecolor='white')
            plt.title(col)
            plt.xlabel(col)
            plt.ylabel('Frequency')

    plt.suptitle('Data Plots', y = 1, fontsize = 24)
    plt.tight_layout()
    plt.show()

In [135]:
def preprocess_nonques_features(df):

    df = rename_features(df)
    df = create_referral_count(df)
    df = clean_client_id(df)
    df = convert_features_to_datetime(df)
    df = convert_float_features_to_int(df)
    df = map_features(df)
    df = one_hot_encode_features(df)
    df = convert_to_int_features(df)

    return df

2.3 Questionnaire Data Preprocessing

In [136]:
def preprocess_ques_features(df):

    # convert all variables to float
    for col in df.columns[27:]:
        df[col] = pd.to_numeric(df[col], errors = 'coerce') # (I think this also removes '.')

    item_cols = df.columns.str.contains('Item').tolist()
    item_cols = df.columns[item_cols]
    for col in item_cols:
        df[col] = df[col].apply(lambda x: x if pd.isna(x) or x.is_integer() else np.nan) # NaN non-integer values
        df[col] = df[col].astype('Int64') # convert to int

    thresh_cols = df.columns.str.contains('Threshold').tolist()
    thresh_cols = df.columns[thresh_cols]
    for col in thresh_cols:
        df[col] = df[col].apply(lambda x: x if pd.isna(x) or x.is_integer() else np.nan)
        df[col] = df[col].astype('Int64')

    total_cols = df.columns.str.contains('Total').tolist()
    total_cols = df.columns[total_cols]
    for col in total_cols:
        df[col] = df[col].apply(lambda x: x if pd.isna(x) or x.is_integer() else np.nan)
        df[col] = df[col].astype('Int64')

    # bool cols (all)
    bool_cols = df.select_dtypes(include = bool)
    for col in bool_cols:
        df[col] = df[col].astype('Int64')

    return df


2 Cleaning

In [137]:
def Clean_Data(df):

    # clean non-questionnaire data
    df = preprocess_nonques_features(df)

    # clean questionnaire data
    df = preprocess_ques_features(df)

    return df

## 1.3 EDA Functions

3 EDA Plots

In [138]:
def single_summary_plot(df, tic_freq):

    statistics = df.describe()

    plt.figure(figsize=(6, 4))
    for index, row in statistics.iterrows():
        plt.plot(statistics.columns, row, label=index)

    plt.xlabel('Column')
    plt.ylabel('Value')
    plt.title('Summary Statistics')
    plt.legend(loc = 'lower right')
    plt.xticks(statistics.columns[::tic_freq])
    plt.show()

In [139]:
def summary_plot(dfs, dfs_names):

    # subplot dimensions
    num_dfs = len(dfs)
    num_rows = num_dfs // 2 + num_dfs % 2

    # plot
    plt.figure(figsize=(15, 5 * num_rows))
    for i, df in enumerate(dfs):

        statistics = df.describe()

        plt.subplot(num_rows, 2, i+1)
        for index, row in statistics.iterrows():
            plt.plot(statistics.columns, row, label=index)

        plt.xlabel('Column')
        plt.ylabel('Value')
        plt.title(f'{dfs_names[i]} Summary')
        plt.xticks(statistics.columns[::2])
        plt.legend(loc = 'center right')

    plt.tight_layout()
    plt.show()

## 1.4 Processing Functions

### 4.1 Duplicates

In [140]:
def find_duplicates_features(df):
    dupe_cols = np.transpose(df).duplicated()
    dupe_cols = df.columns[dupe_cols].tolist()
    dupes_of = {}
    for col_name in dupe_cols:
        col_values = df[col_name]
        dupes = [other_col for other_col in df.columns if (other_col != col_name) and df[other_col].equals(col_values)]
        dupes_of[col_name] = dupes
        print(f'{col_name} is a duplicate of: {dupes}')

Final

In [141]:
def drop_duplicate_features(df):
    dupe_cols = np.transpose(df).duplicated()
    dupe_cols = df.columns[dupe_cols].tolist()
    df = df.drop(columns = dupe_cols)
    return df

### 4.2 Outliers

In [142]:
def column_contents(df):
    pot_cols = []
    for col in df.columns:
        if not df[col].isin([0, 1, pd.NA]).all():
            pot_cols.append(col)
    if len(pot_cols) == 0:
        print(f'Columns only contain 0, 1 and <NA>')
    else:
        print(f'Columns contining more than 0, 1 and <NA>: {pot_cols}')
    return pot_cols

In [143]:
def mad(df):
    median = df.median()
    deviations = np.abs(df - median)
    mad_val = deviations.median() # MAD
    return mad_val

def modified_zscore(df, threshold = 3.5):
    median = df.median()
    mad_val = mad(df) # MAD
    modified_zscores = 0.6745 * (df - median) / mad_val # modified Z-score
    return np.abs(modified_zscores) > threshold

def find_outlier_cols(df, threshold = 3.5): # counts also

    outlier_df = modified_zscore(df)
    outlier_cols = set()
    for col in outlier_df.columns:
        outlier_list = []
        for i, val in outlier_df[col].items():
            if pd.notna(val) and val:
                outlier_list.append(df.iloc[i][col])
        if outlier_list:
            outlier_cols.add(col)
            print(f'{col} outlier count: {len(outlier_list)}')

    # return outlier_df
    outlier_cols = list(outlier_cols)
    outlier_df = df[outlier_cols]
    return outlier_df

In [144]:
def plot_outlier_cols(outlier_df, title, legend=True):
    outlier_df.plot(figsize = (5, 3))
    plt.title(title)
    plt.xlabel('Index')
    plt.ylabel('Total Score')
    if legend:
        plt.legend(loc='upper right')
    else:
        plt.gca().legend().set_visible(False)
    plt.show()

def plot_datetime_features(df):
    plt.figure(figsize=(10, 4))
    for i, col in enumerate(datetime_cols):
        plt.subplot(1, 2, i + 1)
        df[col].hist(bins = 100, color = 'grey', edgecolor = 'white')
        plt.title(col)
        plt.xlabel('Date')
        plt.ylabel('Frequency')
        plt.grid(False)
    plt.tight_layout()
    plt.show()

def plot_outlier_cols2(outlier_df, title, color):

    plot_cols = 4
    plot_rows = len(outlier_df.columns)//4 + 1
    plt.figure(figsize = (plot_cols*3, plot_rows*3))

    for i, col in enumerate(outlier_df.columns, start = 1):
        plt.subplot(plot_rows, 4, i)
        outlier_df[col].plot(color=color)
        plt.title(col)
        plt.xlabel('Index')
        plt.ylabel('Total Score')

    plt.suptitle(title, y = 1, fontsize = 24)
    plt.tight_layout()
    plt.show()

Final

In [145]:
def replace_ques_outliers(df): # shortens search using quartiles

    item_cols = [col for col in df.columns if 'Item' in col]
    item_df = df[item_cols]

    total_cols = [col for col in df.columns if 'Total' in col]
    total_df = df[total_cols]

    # replace negative totals with nan
    for col in total_df:
        df.loc[df[col] < 0, col] = np.nan # replace in df
        total_df.loc[total_df[col] < 0, col] = np.nan # replace in total_df

    # find col contents
    def create_pot_cols(df):
        pot_cols = []
        for col in df.columns:
            if not df[col].isin([0, 1, pd.NA]).all():
                pot_cols.append(col)
        return pot_cols

    # detect whether cols are just 0,1,na
    item_pot_cols = create_pot_cols(item_df)
    total_pot_cols = create_pot_cols(total_df)

    # potential columns containing outliers (shortening the search, only these can contain outliers)
    total_pot_df = total_df[total_pot_cols]
    item_pot_df = item_df[item_pot_cols]

    # outlier counter and df function
    def create_outlier_df(df, threshold = 3.5):

        outlier_df = modified_zscore(df)
        outlier_cols = set()
        for col in outlier_df.columns:
            outlier_list = []
            for i, val in outlier_df[col].items():
                if pd.notna(val) and val:
                    outlier_list.append(df.iloc[i][col])
            if outlier_list:
                outlier_cols.add(col)

        # return outlier_df
        outlier_cols = list(outlier_cols)
        outlier_df = df[outlier_cols]
        return outlier_df

    # find outliers in variables
    total_outlier_df = create_outlier_df(total_pot_df)
    item_outlier_df = create_outlier_df(item_pot_df)

    # replace outliers with nan
    for col in total_outlier_df:
        df.loc[df[col] > 200, col] = np.nan
        total_outlier_df.loc[total_outlier_df[col] > 200, col] = np.nan

    for col in item_outlier_df:
        df.loc[df[col] > 50, col] = np.nan
        item_outlier_df.loc[item_outlier_df[col] > 50, col] = np.nan

    df.loc[df['Total14'] > 37, 'Total14'] = np.nan
    df.loc[df['Item70'] > 3, 'Item70'] = np.nan
    df.loc[df['Item132'] > 5, 'Item132'] = np.nan

    return df


In [146]:
def replace_outliers(df):
    df.loc[df['AgeAtReferralRequest'] == 0, 'AgeAtReferralRequest'] = np.nan
    df = replace_ques_outliers(df)
    return df

### 4.3 Constant and Quasi Constant Features

In [147]:
def drop_const_features(df):
    const_cols = [col for col in df.columns if df[col].nunique() == 1]
    df = df.drop(columns = const_cols)
    return df

In [148]:
def plot_datetime_features(df):
    plt.figure(figsize = (10,4))
    for i, col in enumerate(datetime_cols, 1):
        plt.subplot(1, len(datetime_cols), i)
        plt.hist(df[col], bins = 20, color = 'grey', edgecolor = 'white')
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.title(col)
    plt.tight_layout()
    plt.show()

Final

In [149]:
def quasi_percentage(df):

    # ordinal variables
    ordinal_df = df.select_dtypes(include = ['int64', 'Int64'])

    # select col modes
    modes = ordinal_df.mode() # mode of each col
    modes = modes.iloc[0] # select modes only

    # calc quasi percentage
    quasi_percentages = (ordinal_df == modes).sum() / len(ordinal_df) # how quasi a col is

    return quasi_percentages

# drop quasi features
def drop_quasi_features(df, threshold):

    # calculate quasi percentage
    quasi_percentages = quasi_percentage(df)

    # 0.995 is about 3 observations
    exceeding_threshold_columns = quasi_percentages[quasi_percentages > threshold].index
    df = df.drop(columns = exceeding_threshold_columns)
    return df

### 4.4 Correlation

In [150]:
def select_future_features(df):
    EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
    #EndDescShort_cols = [col for col in df.columns if 'EndDescShort' in col] # none
    future_vars = df.drop(columns = ['ReliableChangeDesc', 'Recovery', 'ReliableRecovery'] + EndDesc_cols)
    return future_vars

In [151]:
def generate_corr_matricies(df, load_matrices):

    file_path = '/content/drive/MyDrive/Data/Dissertation_Data/'

    if not load_matrices:
        # select explanatory variables, removing info on future
        future_df = select_future_features(df)

        # create correlation matrices
        kendall_corr = future_df.corr(method='kendall').abs()
        spearman_corr = future_df.corr(method='spearman').abs()
        kendall_corr.to_csv(file_path + 'kendall_corr.csv') # save
        spearman_corr.to_csv(file_path + 'spearman_corr.csv') # save

    else:
        kendall_corr = pd.read_csv(file_path + 'kendall_corr.csv', index_col=0)
        spearman_corr = pd.read_csv(file_path + 'spearman_corr.csv', index_col=0)

    return kendall_corr, spearman_corr

In [152]:
def plot_correlation_matrices(corr_matricies):

    plt.figure(figsize=(8, 3))

    plt.subplot(121)
    sns.heatmap(corr_matricies[0], annot = False, cmap = 'flare', fmt = ".2f", xticklabels = False, yticklabels = False)
    plt.title('Kendall Correlation Matrix')

    plt.subplot(122)
    sns.heatmap(corr_matricies[1], annot = False, cmap = 'flare', fmt = ".2f", xticklabels = False, yticklabels = False)
    plt.title('Spearman Correlation Matrix')

    plt.tight_layout()
    plt.show()

In [153]:
# count correlated variable pairs above each threshold
def count_correlated_pairs(correlation_matrix, thresholds):

    counts = {}
    for threshold in thresholds:
        correlated_vars = (correlation_matrix.abs() > threshold)
        # get upper triangle
        np.fill_diagonal(correlated_vars.values, False)
        upper_triangle = correlated_vars.values[np.triu_indices_from(correlated_vars, k=1)]
        count = np.sum(upper_triangle)
        counts[threshold] = count
    return counts

def correlated_features_above_treshold(matricies, thresholds):

    print('Kendall Correlation:')
    kendall_counts = count_correlated_pairs(matricies[0], thresholds)
    for threshold, count in kendall_counts.items():
        print(f'{threshold * 100}%: {count}')

    print('Spearman Correlation:')
    spearman_counts = count_correlated_pairs(matricies[1], thresholds)
    for threshold, count in spearman_counts.items():
        print(f'{threshold * 100}%: {count}')


In [154]:
def drop_corr_cols(df, corr_matrix, threshold, ignore=True):

    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    corr_cols = [col for col in upper_tri.columns if any(upper_tri[col] > threshold)]

    try:
        df = df.drop(corr_cols, axis=1)
    except KeyError as e:
        if ignore:
            #print(f"Ignoring KeyError: {e}")
            pass
        else:
            raise

    return df

Final

In [155]:
def remove_corr_features(df, load_matrices, threshold):

    kendall_corr, spearman_corr = generate_corr_matricies(df, load_matrices)

    # remove highly correlated variables
    df = drop_corr_cols(df, kendall_corr, threshold)
    df = drop_corr_cols(df, spearman_corr, threshold)

    return df


### 4.5 Missing Values within Columns

In [156]:
def drop_missing_values_col(df, threshold):

    missing_value_percentage = df.isna().mean(axis=0) # cols

    drop_cols = missing_value_percentage[missing_value_percentage > threshold].index
    df = df.drop(drop_cols, axis=1)

    return df

### 4.6 Missing Values within Rows

In [157]:
def drop_missing_values_row(df, threshold):

    missing_value_percentage = df.isna().mean(axis=1) # rows

    drop_cols = missing_value_percentage[missing_value_percentage > threshold].index
    df = df.drop(drop_cols, axis=0)

    return df

### 4.7 Scaling

In [158]:
def standardise(df):
    scaler = StandardScaler()
    df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return df

### 4.8 Imputation

In [159]:
def impute_data(df, method, n_neighbours=None, max_iter=None):

    original_dtypes = df.dtypes.to_dict()

    #EndDescShort_cols = [col for col in df.columns if 'EndDescShort' in col] # none
    EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
    nontarget_df = df.drop(columns = ['ReliableChangeDesc', 'Recovery', 'ReliableRecovery'] + EndDesc_cols)

    datetime_cols = nontarget_df.select_dtypes(include=['datetime64[ns]']).columns
    for col in datetime_cols:
        nontarget_df[col] = pd.to_numeric(nontarget_df[col])

    if method == 'none':
        nontarget_mat = nontarget_df.values

    elif method == 'mean':
        imputer = SimpleImputer(strategy='mean')
        nontarget_mat = imputer.fit_transform(nontarget_df) # *numpy array*

    elif method == 'median':
        imputer = SimpleImputer(strategy='median')
        nontarget_mat = imputer.fit_transform(nontarget_df)

    elif method == 'knn':
        imputer = KNNImputer(n_neighbors=n_neighbours)
        nontarget_mat = imputer.fit_transform(nontarget_df)

    elif method == 'iterative':
        imputer = IterativeImputer(max_iter=max_iter, random_state=2001)
        nontarget_mat = imputer.fit_transform(nontarget_df)

    nontarget_df.iloc[:, :] = nontarget_mat.tolist()

    # add imputed data to df
    for col in nontarget_df.columns:
        df[col] = nontarget_df[col]

    if method == 'none':
        pass

    else:
        for col, dtype in original_dtypes.items():
            if dtype == 'datetime64[ns]':
                pass # keep as float
            else:
                try:
                    df[col] = df[col].astype(dtype)
                except (ValueError, TypeError):
                    # *convert to original dtype*
                    df[col] = df[col].round().astype(int)
                    df[col] = df[col].astype(dtype)

    return df


In [160]:
# def multi_impute_data(df, method='knn', n_neighbours=3, iter=1):

#     EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
#     #EndDescShort_cols = [col for col in df.columns if 'EndDescShort' in col] # none
#     nontarget_df = df.drop(columns = ['ReliableChangeDesc', 'Recovery', 'ReliableRecovery'] + EndDesc_cols)

#     datetime_cols = nontarget_df.select_dtypes(include=['datetime64[ns]']).columns
#     for col in datetime_cols:
#         nontarget_df[col] = nontarget_df[col].astype(int) / 10**9

#     ordinal_cols = list(nontarget_df.select_dtypes(include=['Int64', 'int64']).columns)
#     imputed_dfs = []

#     for i in range(iter):

#         copy_df = nontarget_df.copy()

#         if method == 'mean':
#             mean_imputer = SimpleImputer(strategy='mean')
#             copy_df[nontarget_df.columns] = mean_imputer.fit_transform(copy_df[nontarget_df.columns])

#         elif method == 'median':
#             median_imputer = SimpleImputer(strategy='median')
#             copy_df[nontarget_df.columns] = median_imputer.fit_transform(copy_df[nontarget_df.columns])

#         elif method == 'knn':
#             KNN_imputer = KNNImputer(n_neighbors=n_neighbours)
#             copy_df[nontarget_df.columns] = KNN_imputer.fit_transform(copy_df)

#         imputed_dfs.append(copy_df)

#     df = np.mean(imputed_dfs, axis=0)
#     print(df.dtypes)
#     for col in datetime_cols:
#         df[col] = pd.to_datetime(df[col], unit='ns')

#     # convert imputed values for ordinal variables to discrete values
#     for col in ordinal_cols:
#         df[col] = df[col].round().astype(int)

#     return df


### 4.9 Feature Engineering

In [161]:
def create_date_features(df):

    # df['ReferralYear'] = df['ReferralDate'].dt.year
    # df['ReferralMonth'] = df['ReferralDate'].dt.month
    # df['ReferralWeek'] = df['ReferralDate'].dt.isocalendar().week
    # df['ReferralDay'] = df['ReferralDate'].dt.day
    # df['ReferralHour'] = df['ReferralDate'].dt.hour
    # df['ReferralWeekDay'] = df['ReferralDate'].dt.dayofweek
    # df['ReferralYearDay'] = df['ReferralDate'].dt.dayofyear

    df['YearofQuestionnaire'] = df['DateOfQuestionnaire'].dt.year
    df['MonthofQuestionnaire'] = df['DateOfQuestionnaire'].dt.month
    df['WeekofQuestionnaire'] = df['DateOfQuestionnaire'].dt.isocalendar().week
    df['DayofQuestionnaire'] = df['DateOfQuestionnaire'].dt.day
    df['HourofQuestionnaire'] = df['DateOfQuestionnaire'].dt.hour
    df['WeekDayofQuestionnaire'] = df['DateOfQuestionnaire'].dt.dayofweek
    df['YearDayofQuestionnaire'] = df['DateOfQuestionnaire'].dt.dayofyear

    return df

In [162]:
def feature_engineering(df):

    df = create_date_features(df)

    return df

### 4.10 Feature Importance

In [163]:
def find_important_features(df, k_features, target='Recovery'):

    """
    k is the number of features each feature selection method should select.
    NOT the number of features returned

    """

    if k_features == None:
        important_features = df.columns

    else:
        features = df.dropna()
        EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
        explanatory = features.drop(['Recovery', 'ReliableRecovery', 'ReliableChangeDesc'] + EndDesc_cols, axis = 1)
        target = features[target]

        # fishers score
        kbest_fisher = SelectKBest(score_func=f_classif, k=k_features)
        selected_fisher = kbest_fisher.fit_transform(explanatory, target)
        index_fisher = kbest_fisher.get_support(indices=True)
        names_fisher = explanatory.columns[index_fisher]

        # mutual information gain
        def mutual_info_classif_wseed(X, y):
            return mutual_info_classif(X, y, random_state=11)
        kbest_gain = SelectKBest(score_func=mutual_info_classif_wseed, k=k_features)
        selected_gain = kbest_gain.fit_transform(explanatory, target)
        index_gain = kbest_gain.get_support(indices=True)
        names_gain = explanatory.columns[index_gain]

        important_combined = set(names_fisher).union(set(names_gain))
        important_features = list(important_combined)

    return important_features

In [164]:
def select_important_features(df, k_features, target='Recovery'):

    important_features = find_important_features(df, k_features, target='Recovery')

    EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
    df = df[['Recovery', 'ReliableRecovery', 'ReliableChangeDesc'] + EndDesc_cols + important_features]

    return df

### 4 Prepare Data

In [165]:
def Prepare_Data(df,
                 quasi_thresh=1.0,
                 corr_thresh=1.0,
                 load_matrices=True,
                 col_thresh=1.0,
                 row_thresh=1.0,
                 imputation_method='knn',
                 n_neighbours=10,
                 max_iter=10,
                 k_features=200):

    df = drop_duplicate_features(df)
    df = replace_outliers(df)
    df = drop_const_features(df)
    df = drop_quasi_features(df, quasi_thresh)
    df = remove_corr_features(df, corr_thresh, load_matrices)
    df = drop_missing_values_col(df, col_thresh)
    df = drop_missing_values_col(df, row_thresh)
    #df = standardise(df)
    df = impute_data(df, imputation_method, n_neighbours, max_iter)
    #df = feature_engineering(df)
    df = select_important_features(df, k_features)

    return df

## 1.5 Modelling

### Summary Plots

In [166]:
def ModelSelection_Summary(model):

# pr curve

    scores = model[0]
    preds = model[1]
    actuals = model[2]

    print('Average accuracy score: {0}'.format(np.average(scores)))

    prec, recall, _ = metrics.precision_recall_curve(actuals, preds)
    print('AUPRC score: {0}\n'.format(metrics.auc(recall, prec)))

    # plot pr curve
    plt.figure(figsize=(12,4))
    plt.subplot(121)
    plt.plot(recall, prec, marker='.')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.show()

# confusion matrix

    # probabilities to binary
    threshold = 0.5
    binary_preds = [1 if pred >= threshold else 0 for pred in preds]

    # plot confusion matrix
    conf_matrix = confusion_matrix(actuals, binary_preds)

    plt.subplot(121)
    sns.heatmap(conf_matrix, annot = True, fmt = 'd', cmap = 'Purples')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.show()

### XGBoost

In [167]:
def XGBoost_ModelSelection(df, target, selector, param_grid, k=5):

    # dataset
    sample = df.dropna(subset = [target])
    EndDesc_cols = [col for col in df.columns if 'EndDesc' in col] #EndDescShort_cols = [col for col in df.columns if 'EndDescShort' in col] # none
    X = sample.drop(['ReliableChangeDesc', 'ReliableRecovery', 'Recovery'] + EndDesc_cols, axis = 1)
    y = sample[target]
    cols = X.columns

    # machine learning algorithm
    classifier = XGBClassifier()

    # feature selection method
    if selector == 'SelectFromModel':
        selector = SelectFromModel(classifier)
    elif selector == 'RFE':
        selector = RFE(classifier)
    else:
        # fill this #
        raise ValueError('Unsupported selector type')

    # pipeline
    pipeline = Pipeline([("FS", selector), ("classifier", classifier)])

    # initialise lists
    scores, preds, actuals = [], [], []

    # cross validation and hyperparameter tuning
    outer_cv = StratifiedKFold(n_splits = k, shuffle = True)
    inner_cv = StratifiedKFold(n_splits = k, shuffle = True) # both set to k for now
    for train_index, test_index in outer_cv.split(X, y):

        # outer CV train and test sets
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # hyper-parameter tuning
        grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=inner_cv, scoring='accuracy', verbose=0, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        print("Inner CV accuracy: {}".format(grid_search.best_score_)) # validation sets

        # optimal model
        estimator = grid_search.best_estimator_

        # count features selected
        support = estimator.named_steps['FS'].get_support()
        num_feat = np.sum(support)
        print("Number of selected features {0}".format(num_feat))

        # features selected
        col_index = np.where(support)[0]
        col_names = [cols[col] for col in col_index]
        print("Selected features {0}".format(col_names))

        # hyperparameters selected
        print("Max depth {0}".format(estimator.named_steps["classifier"].max_depth))
        print("Number of trees {0}".format(estimator.named_steps["classifier"].n_estimators))
        print("Learning rate {0}".format(estimator.named_steps["classifier"].learning_rate))
        print("Minimum child weight {0}".format(estimator.named_steps["classifier"].min_child_weight))
        print("Subsample {0}".format(estimator.named_steps["classifier"].subsample))
        print("Colsample bytree {0}".format(estimator.named_steps["classifier"].colsample_bytree))
        print("Gamma {0}".format(estimator.named_steps["classifier"].gamma))
        print("Lambda {0}".format(estimator.named_steps["classifier"].reg_lambda))
        print("Alpha {0}".format(estimator.named_steps["classifier"].reg_alpha))

        # evaluating optimised model on test
        predictions = estimator.predict(X_test)
        score = metrics.accuracy_score(y_test, predictions)
        scores.append(score)
        print('Outer CV accuracy: {}'.format(score)) # test sets

        print("--------------------------------------------------")

        probs = estimator.predict_proba(X_test)[:, 1]
        preds.extend(probs)
        actuals.extend(y_test)

    return scores, preds, actuals


## BERT

In [168]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import torch

class BERTClassifier(BaseEstimator, TransformerMixin):
    def __init__(self, model_name='bert-base-uncased', max_length=128):
        self.model_name = model_name
        self.max_length = max_length
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(model_name)

    def fit(self, X, y):
        self.model.train()
        return self

    def predict(self, X):
        inputs = self.tokenizer(X.tolist(), return_tensors='pt', truncation=True, padding=True, max_length=self.max_length)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        return preds.numpy()

    def predict_proba(self, X):
        inputs = self.tokenizer(X.tolist(), return_tensors='pt', truncation=True, padding=True, max_length=self.max_length)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        return probs.numpy()

def BERT_ModelSelection(df, text_column, target, param_grid, k=5):
    # dataset to model
    sample = df.dropna(subset=[target])  # remove missing target values
    X = sample[text_column]
    y = sample[target]

    # machine learning model
    classifier = BERTClassifier()

    # pipeline
    pipeline = Pipeline([('classifier', classifier)])

    # initialize lists
    scores = []
    preds = []
    actuals = []

    # k-fold CV
    kf = StratifiedKFold(n_splits=k, shuffle=True)
    for train_index, test_index in kf.split(X, y):
        # train and test data for CV
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # finding optimal models, hyperparameter tuning
        grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='accuracy', verbose=0, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        print("Internal CV Accuracy of estimator: {}".format(grid_search.best_score_))

        # best estimator
        estimator = grid_search.best_estimator_

        # print hyperparameters selected
        # For BERT, hyperparameters can include learning rate, batch size, etc.
        # Example:
        print("BERT Model used: {}".format(estimator.named_steps["classifier"].model_name))
        print("Max length used: {}".format(estimator.named_steps["classifier"].max_length))

        # predicting the test data with the optimized models
        predictions = estimator.predict(X_test)
        score = accuracy_score(y_test, predictions)
        scores.append(score)
        print('Accuracy performance on this test set: {}'.format(score))

        print("--------------------------------------------------")

        probs = estimator.predict_proba(X_test)[:, 1]
        preds.extend(probs)
        actuals.extend(y_test)

    return scores, preds, actuals